In [1]:
import os 

In [2]:
class Hooklog:
# class SortedAPIList:
    path = None
    par = None
    rval = None
    
    digitname = None
    length = None
    hookli = None
    
    def __init__(self, path, par, rval):
        self.path = path
        self.par = par
        self.rval = rval
        self.hookli = list()
        self.length = 0
        
        self.__parseDigitname()
        self.__parseHooklog()
    
    def isExist(self):
        if os.path.isfile(self.path):
            return True
        else:
            return False
    
    def __len__(self):
        return self.length

    def __iter__(self):
        return iter(self.hookli)
    
    def __getitem__(self, key):
        return self.hookli[key]
    
    def __str__(self):
        return "class Hooklog, %s, par = %d, len = %d, digit name = %s" % (self.path, self.par, 
                                                                           self.length, self.digitname)
    
    # private functions
    def __parseDigitname(self):
        if self.path != "":
            self.digitname = self.path.split('/')[-1][:6] + '-' + self.path.split('_')[-1].split('.')[0]
    
    
    def __parseHooklog(self):
        if self.isExist():
            handle = open(self.path, 'rb')     # MIKE: change mode from 'r' to 'rb',
                                               # because windows cannot handel seek() in text(r) mode
            while 1:
                (tick, api) = self.__getNextPair(handle)
                if tick == 0: break # end of file or fail to read
                self.hookli.append((tick, api))
                self.length += 1    
            if handle: handle.close()
            #if logtype == 'H':
            self.hookli.sort(key = lambda tup: int(tup[0])) # sort by tick
            #self.hookli =  filter(lambda (tick, api): api != '', self.hookli) #=====2/17 python2
            self.hookli =  list(filter(lambda tup: tup[1] != '', self.hookli)) #=====7/10 python3
            
        else:
            print (self.path, ' is not exist')
            pass
        
    def __getNextPair(self, handle):
        tick = 0
        api = 'api'
        
        while 1:
            line = handle.readline().decode('windows-1252').strip()
            if not line: break # end of file
            if line[0] == '#': # start a new call 
                # and logtype == 'H' form Mr.Li
                tick = line[1:]
                api =  handle.readline().decode('windows-1252').strip()
                
                # MIKE: comment 20150811
                #if par == 1 and (api.startswith('Reg') or api == 'LoadLibrary' or api.startswith('=Reg')):
                #    api += handle.readline().strip()
                # MIKE: 20150811 new for Transformer
                # hack=> if api.startswith('=Reg'): api = api[1:]
                if self.par == 1 or self.rval == 1:
                    api = self._getParValue(api, handle)
                break
            else:
                continue
        
        return (tick, api)

    # protected functions
    # MIKE: 20160307, basic par. I move it out for better creating new class FeatureHooklog. 
    #                            It has to be protected!
    def _getParValue(self, api, handle):
        if api.startswith('Reg') or api == 'LoadLibrary':
            api += handle.readline().decode('windows-1252').strip()
        return api
    
    # public functions
    def getWinSet(self, win):
        li_set = set()
        for i in range(self.length - win + 1): 
            key = ''
            for w in range(win):               
                key += self.li[i+w][1] 
            li_set.add(key)
        return li_set

In [3]:
class FeatureHooklog(Hooklog):
    
    isAll = None #check if consider all api 
    
    def __init__(self, path ,isAll):
        self.isAll = isAll
        Hooklog.__init__(self, path, 1, 1)
        

    def __str__(self):
        return "class FeatureHooklog, %s, par = %d, len = %d, digit name = %s" % (
            self.path, self.par, self.length, self.digitname)
    
    
    # protected functions
    # MIKE: 20160307, in FeatureHooklog, new api is (api,firstpar,return)
    def _getParValue(self, api, handle):          
        parword = "#PR#"
        retword = "#PR#"
    
        if api not in API: 
            if api in passAPI: 
                return ''
            else:
                #print api, "not in API"
                return api
        
        while 1:
            pos = handle.tell()
            line = handle.readline().decode('windows-1252').strip()
          
            if not line: # reach to the end of file
                return api+parword+retword
            if line[0] == '#': # reach to next call
                handle.seek(pos)
                return api+parword+retword
        
            delimiter = line[1:].find('=') +1 # MIKE: for =Reg
            p = line[:delimiter]
            v = line[delimiter+1:].strip() #===value
        
            if not self.isAll:       
                if self.par == 1 and (p in APIParRet[api][0]):
                    parword += self.__parTrans(api, p, v, 1) # v sometimes could be empty!
                if self.rval == 1 and (p in APIParRet[api][1]):
                    retword += v
            else:
                if p in ['Return', 'result']:
                    retword += v
                else:
                    parword = parword + line + '?' 
                    
    # private functions
    def __parTrans(self, api, key, value, trans):
        t = api+key
            
        if not trans: return value #don't need trans
        
        FuncDict = self.__funcdict()
        
        if t in FuncDict:
            return FuncDict[t](value) # function dictionary
        else:
            #print "no such transformation", api, key
            return value
        
    def __libTrans(self, value):
        DIR = LIB = EXT = "non"
        
        if value == "": return "NON@NON@NON"
        
        lvalue = value.lower()
        tokens = value.split('\\')
        
        # DIR
        if lvalue[1] != ':':
            DIR = "SYS"
        elif lvalue[0] == '\\':
            DIR = "LOC"
        else:
            for d in dir_dict:
                if d in lvalue:
                    DIR = dir_dict[d]
                    break

        # EXT
        t = tokens[-1].split('.')
        ext = t[-1]
        if ext == "" or len(t) == 1: # extension is . or no extension
            EXT = "DLL"
        else:
            EXT = ext.upper()
            
        # LIB
        LIB = tokens[-1].split('.')[0].lower()
    
        #return DIR+"@"+LIB+"@"+EXT+"("+value+")"
        return DIR+"@"+LIB+"@"+EXT

    def __fileDesiredAcc(self,value):
        return '&&dA:'+ value.replace(' ',';')
    
    def __fileTrans(self, value):
        DIR = EXT = ""
    
        # for empty value
        if value == "":return "NON@NON"
    
        lvalue = value.lower().replace('/', '\\')
        tokens = lvalue.split('\\')
    
        #DIR
        if lvalue[:4] == ("\\\\.\\"):
            DIR = lvalue
        elif lvalue == 'conin$' or lvalue == 'conout$':
            DIR = 'CONSOLE'
        elif lvalue[0] == '\\' or len(tokens) == 1:
            DIR = "LOC"
        else:
            for d in dir_dict:
                if d in lvalue:
                    DIR = dir_dict[d]
                    break
        if DIR == "": DIR = "ARB" # possbily c:
    
        # EXT
        t = tokens[-1].split('.')
        ext = t[-1]
        if ext == "" or len(t) == 1: # extension is . or no extension
            EXT = "NON"
        else:
            EXT = ext.upper()
    
        #return DIR+"@"+FIL+"@"+EXT+"("+value+")"
        return DIR+"@"+EXT
    
    def __keyTrans(self, value):
        HK = KEY = ""
        if value == "": return "NON@NON"
    
        lvalue = value.lower()
        tokens = lvalue.split('\\')
    
        # HK
        if tokens[0] in hkey_dict:
            HK = hkey_dict[tokens[0]]
        else:
            HK = "SUBK"
        
        # KEY
        subkey = lvalue[lvalue.find('\\'):]
        KEY = self.__remove_reg(subkey)
        KEY = self.__remove_fileName(KEY)
        KEY = self.__remove_mshist(KEY)
        KEY = self.__remove_sessioninfo(KEY)
    
        #return HK+"@"+KEY+"("+value+")"
        return HK+"@"+KEY
    
    def __keySupPathTrans(self, value):
        keyBasic = self.__keyTrans(value).split('@')
        if( len(keyBasic)!= 2):
            print ('key rule error!')
        else:
            HK = keyBasic[0]
            KEY = keyBasic[1]
            try:
                KEY = subPathstrim(KEY)
            except:
                KEY = KEY
                #print KEY # for debug
        return HK+"@"+KEY
     
    def __thdTrans(self, value):
        return "NON"
    
    def __execTrans(self, value):
        return self.__fileTrans(value)

    def __cmdTrans(self, value):
        return value

    
    def __remove_reg(self, key):
        rvalue = ""
        tokens = key.split('\\')

        for t in tokens:
            if t.count('-') > 3:
                rvalue += "\\{REG}"
            else:
                rvalue += ('\\'+t)

        return rvalue

    def __remove_fileName(self, key):
        tok = key.split('\\')
        tok = [t for t in tok if t!='']
        for i,t in enumerate(tok):
            if '.' in t and len(t.split('.')) == 2 and len(t.split(' '))==1: 
                tok[i] = '.'+t.split('.')[1]
        return '\\'.join(tok)

    def __remove_mshist(self, key):
        tok = key.split('\\')
        tok = [t for t in tok if t!='']
        for i,t in enumerate(tok):
            if 'mshist' in t:
                tok[i] = 'mshistDate'
        return '\\'.join(tok)  

    def __remove_sessioninfo(self, key):
        tok = key.split('\\')
        parse_str = '\\'.join(tok)
        if 'software\\microsoft\\windows\\currentversion\\explorer\\sessioninfo' in parse_str:
            tok = parse_str.split('\\')
            if len(tok) == 7:
                tok[-1] = 'SESSIONID'
        return '\\'.join(tok)

    def __funcdict(self):
        FuncDict = {"LoadLibrarylpFileName": self.__libTrans,
                    "CreateProcessInternallpApplicationName": self.__execTrans,
                    "WinExecCmdLine": self.__cmdTrans,
                    "CreateThreaddwThreadId": self.__thdTrans, 
                    "CreateFilehName": self.__fileTrans,
                    "CreateFiledesiredAccess": self.__fileDesiredAcc, # NEW 
                    "CopyFilelpExistingFileName": self.__fileTrans,
                    "DeleteFilefileName": self.__fileTrans,
                    "RegQueryValuehKey": self.__keySupPathTrans,
                    "RegEnumValuehKey": self.__keySupPathTrans,
                    "RegOpenKeyhKey": self.__keySupPathTrans,
                    "RegCloseKeyhKey": self.__keySupPathTrans,
                    "RegSetValuehKey": self.__keySupPathTrans,
                    "RegCreateKeyhKey": self.__keyTrans,
                    "=RegDeleteKeyhKey": self.__keyTrans, #####
                    "RegDeleteKeyhKey": self.__keyTrans,
                    "RegDeleteValuehKey": self.__keyTrans
        }
        return FuncDict
    
    # public functions
    def getWinSet(self, win):
        li_set = set()
        for i in range(self.length - win + 1):
            key = list()
            for w in range(win):
                key.append(self.hookli[i+w][1]) # append (api, v, r)
            li_set.add(tuple(key))
        return li_set
    
    def getHkli_containTS(self):
        return self.hookli
    
    def getHkli_noContainTS(self):
        hookli_noTS = list()
        for i, (ts, api) in enumerate(self.hookli):
            hookli_noTS.append(api)
        
        return hookli_noTS
    
dir_dict = {
    "\\windows\\system32\\": "SYS",
    "\\windows\\system\\": "SYS",
    "\\program files\\": "PRO",
    #"\\windows\\": "WIN",
    #"\\documents and settings\\all users\\": "USR",
    "\\documents and settings\\": "USR",
    "\\docume~1\\": "USR",
    "\\windows\\temp\\": "TMP",
    ":\\temp\\": "TMP"
}

hkey_dict = {
    "hkey_classes_root": "HKCR",
    "hkey_current_user": "HKCU",
    "hkey_local_machine": "HKLM",
    "hkey_users": "HKUS",
    "hkey_current_config": "HKCC"
}

APIParRet = {
    # LIB
    "LoadLibrary": [["lpFileName"], ["Return","result"]],
    # PROC
    "CreateProcessInternal": [["lpApplicationName"], ["Return","result"]],
    "OpenProcess": [[], ["Return","result"]], # it has procName, but might be empty
    "ExitProcess": [[], ["Return","result"]],
    "WinExec": [["CmdLine"], ["Return","result"]],
    "CloseHandle": [[], ["Return","result"]],
    "CreateRemoteThread": [[], ["Return","result"]], # it has procName, but might be empty
    "TerminateProcess": [[], ["Return","result"]],
    "TerminateThread": [["hThread"], ["Return","result"]],
    "CreateThread": [["dwThreadId"], ["Return","result"]],
    "OpenThread": [["dwThreadId"], ["Return","result"]],
    # FILE
    "CopyFile": [["lpExistingFileName"], ["Return","result"]], # include lpNewFileName?
    "CreateFile": [["hName", "desiredAccess"], ["Return","result"]], #MIKE: actually it is 'lpFileName' in spec.
    "WriteFile": [[], ["Return","result"]],
    "ReadFile": [[], ["Return","result"]],
    "DeleteFile": [["fileName"], ["Return","result"]], #MIKE: actually it is 'lpFileName' in spec.
    # REG
    "RegOpenCurrentUser": [[], ["Return","result"]],
    "RegQueryValue": [["hKey"], ["Return","result"]],
    "RegEnumValue": [["hKey"], ["Return","result"]],
    "RegOpenKey": [["hKey"], ["Return","result"]],
    "RegCloseKey": [["hKey"], ["Return","result"]],
    "RegSetValue": [["hKey"], ["Return","result"]],
    "RegCreateKey": [["hKey"], ["Return","result"]],
    "=RegDeleteKey": [["hKey"], ["Return","result"]], # MIKE: misprint the '=' #============
    "RegDeleteKey": [["hKey"], ["Return","result"]], # MIKE: misprint the '='
    "RegDeleteValue": [["hKey"], ["Return","result"]],
    # NET winhttp.dll
    "WinHttpConnect": [["pswzServerName"], ["Return","result"]],
    "WinHttpCreateUrl": [["wszUrl"], ["Return","result"]],
    "WinHttpOpen": [["pwszUserAgent"], ["Return","result"]],
    "WinHttpOpenRequest": [["pwszObjectName"], ["Return","result"]],
    "WinHttpReadData": [["lpBuffer"], ["Return","result"]],
    "WinHttpSendRequest": [["pwszHeaders"], ["Return","result"]],
    "WinHttpWriteData": [["lpBuffer"], ["Return","result"]],
    "WinHttpGetProxyForUrl": [["lpcwszUrl"], ["Return","result"]],
    # NET winnet32.dll
    "InternetOpen": [["lpszAgent"], ["Return","result"]],
    "InternetConnect": [["lpszServerName"], ["Return","result"]],
    "HttpSendRequest": [["lpszHeaders"], ["Return","result"]],
    "GetUrlCacheEntryInfo": [["lpszUrlName"], ["Return","result"]],
}

API = APIParRet.keys()
passAPI = ['CreateProcess']

## subpath test

In [4]:
# -1:keep, 0:*, 1:1stLayer,10:negative, 20:postivite, 30:include
subPathDict = {
    "control panel": {'desktop': (-1,'desktop')},
    "system": {
        'setup': (-1,'sys_setup'),
        'controlset001':{'control':{'class': (0,'sys_ctlSet001_ctl_class\\*')}},
        'currentcontrolset':{
            'control':{
                'session manager': (0,'sys_curCtlSet_ctl_sessionManager\\*'),
                'mediaproperties': (0,'sys_curCtlSet_ctl_mediaProperties\\*'),
                'productoptions': (0,'sys_curCtlSet_ctl_productoptions\\*')},
            'services':{
                'netbt': (0,'sys_curCtlSet_svc_netbt\\*'),
                'tcpip': (10,'sys_curCtlSet_svc_tcpip', ['max','min','timeout']),
                'dnscache': (10,'sys_curCtlSet_svc_dnscache', ['max','min','timeout']),
                'ldap': (10,'sys_curCtlSet_svc_ldap', ['max','min','timeout']),
                'winsock': (10,'sys_curCtlSet_winsock', ['max','min','timeout']),
                'winsock2':{
                    'parameters':{
                        'protocol_catalog9': (0,'sys_curCtlSet_svc_winsock2_catalog9\\*'),
                        'namespace_catalog5': (0,'sys_curCtlSet_svc_winsock2_catalog5\\*'),
                        'ELSE' : (10,'sys_curCtlSet_svc_winsock2', ['max','min','timeout'])}, #
                    'ELSE' : (10,'sys_curCtlSet_svc_winsock2', ['max','min','timeout'])}, #
                'windows test 5.0': (1,'sys_curCtlSet_svc_winTest5.0')}},
        'ELSE':(-1,'sys')
    },
    'software':{
        'microsoft':{
            'commandprocessor': (0,'soft_ms_commandprocessor\\*'),
            'audiocompressionmanager': (0,'soft_ms_audiocompressionmanager\\*'),
            'multimedia':{
                'audio compression manager': (0,'soft_ms_mulitimedia_audiocompressionmanager\\*'),
                'ELSE' :(20,'soft_ms_multimedia',['audio'])},
            'ole': (0,'soft_ms_ole\\*'),
            'ctf': (0,'soft_ms_ctf\\*'),
            'com3': (0,'soft_ms_com3\\*'),
            'tracing': (1,'soft_ms_tracing'),
            'internet explorer':{'main':{'featurecontrol':(-1,'soft_ms_IE_featureCtl')}},
            'rpc': (10,'soft_ms_rpc', ['max','min','timeout']),
            'wbem':{
                'cimom': (20,'soft_ms_wbem_cimom', ['log']),
                'wmic': (0,'soft_ms_wbem_wmic\\*')},
            'windows nt':{'currentversion':{
                    'languagepack':(0,'soft_ms_winNT_languagepack\\*'),
                    'winlogon':(0,'soft_ms_winNT_winlogon\\*'),
                    'fontlink':(0,'soft_ms_winNT_fontlink\\*'),
                    'fontsubstitutes':(0,'soft_ms_winNT_fontsubstitutes\\*'),
                    'imm':(0,'soft_ms_winNT_imm\\*'),
                    'csdversion':(0,'soft_ms_winNT_csdversion\\*')}},
            'windows':{'currentversion':{
                    'thememanager':(0,'soft_ms_win_thememanager\\*'),
                    'telephony':(0,'soft_ms_win_telephony\\*'),
                    'profilelist':(0,'soft_ms_win_profilelist\\*'),
                    'programfilesdir':(0,'soft_ms_win_programfilesdir\\*'),
                    'commonfilesdir':(0,'soft_ms_win_commonfilesdir\\*'),
                    'explorer':{
                        'user shell folders': (0,'soft_ms_win_explorer_userShellFolder\\*'),
                        'shell folders': (0,'soft_ms_win_explorer_shellFolders\\*'),
                        'ELSE' : (-1,'soft_ms_win_explorer')},
                    'internet settings':{
                        'zones':(0,'soft_ms_win_internetSettings_zones\\*'),
                        'cache':{'paths':(0,'soft_ms_win_internetSettings_cache_paths\\*')},
                        'ELSE':(10,'soft_ms_win_internetSettings',
                                ['max','min','timeout','length','bound','limit','timesecs','range'])},
                    'ELSE':(-1,'soft_ms_win_currentversion')
                }}
        }
    },
    'clsid':{'{REG}':{
            'inprocserver32':(0,'clsid_{REG}_inprocserver32\\*'),
            'appid':(0,'clsid_{REG}_appid\\*')
        }},
    'appid':{'{REG}':(0,'appid_{REG}\\*')},
    'interface':{'{REG}':{'proxystubclsid32':(0,'interface_{REG}_proxystubclsid32\\*')}},
    '{REG}':{
        'environment':(-1,'{REG}_envr'),
        'volatile environment':(-1,'{REG}_volatileEnvr')
    }
}


In [5]:
# -1:keep, 0:*, 1:1stLayer,10:negative, 20:postivite,
def subPathstrim(subpath): # -1, 0, 10, 20
    subFlag = False
    startIndex = 0
    startToken = ''
    returnVal = ''
    
    tokens = subpath.split('\\')
    for i, token in enumerate(tokens):
        if token in subPathDict:
            startIndex = i
            startToken = token
            break
        else:
            return ('\\').join(tokens)

    endIndex = startIndex
    endToken = startToken
    tmpDict = subPathDict
    endFlag = True if type(tmpDict[endToken])==tuple else False
    
    while( not endFlag ): # not to the end 
        tmpDict = tmpDict[endToken]
        endIndex += 1
        endToken = tokens[endIndex]
        if endToken in tmpDict:
            endFlag = True if type(tmpDict[endToken])==tuple else False
        else:
            endToken = 'ELSE'
            endIndex -= 1
            break
    
    #print subPath ####
    subType = tmpDict[endToken][0]
    subString = tmpDict[endToken][1]
    keepFrontString = ('\\').join(tokens[:startIndex])
    
    if subType == 0:
        returnVal = keepFrontString + subString
    elif subType == -1:
        keepEndtString = ('\\').join(tokens[endIndex+1:])
        returnVal = keepFrontString + subString + '\\' + keepEndtString
    elif subType == 1:
        keepEndtString = ('\\').join(tokens[endIndex+1:endIndex+2])
        returnVal = keepFrontString + subString + '\\' + keepEndtString
    elif subType == 10:
        ignoreli = tmpDict[endToken][2]
        keepEndtString = ('\\').join(tokens[endIndex+1:])
        keepFlag = True
        for k in ignoreli:
            if k in keepEndtString:
                returnVal = keepFrontString + subString + '\\*'
                keepFlag = False
                break
        if keepFlag:
            returnVal = keepFrontString + subString + '\\' + keepEndtString
    elif subType == 20:
        keepli = tmpDict[endToken][2]
        keepEndtString = ('\\').join(tokens[endIndex+1:])
        ignoreFlag = True
        for k in keepli:
            if k in keepEndtString:
                returnVal = keepFrontString + subString + '\\'+ k
                ignoreFlag = False
                break
        if ignoreFlag:
            returnVal = keepFrontString + subString + '\\*' 
    
    return returnVal

## example

#### for winnoing rules

In [6]:
try:
    KEY = subPathstrim('control panel\\desktop\\su')
    print (KEY)
except:
    print ("err: ", KEY) # for debug

desktop\su


#### use FeatureHooklog ( filename, isContainTimeStamp)

In [7]:
testDir = "test/hooklogs/"
base = testDir + "ffe9afc0f1344eb2ee6b1c391445bdd0_3152.trace.hooklog"
hk = testDir + "ffe83caf42822dd3c1a3690db5caf740_3156.trace.hooklog"
fp_base = FeatureHooklog(base, 0)
fp_hk = FeatureHooklog(hk, 0)
for i in fp_base:
    print(i)

('231680000', 'LoadLibrary#PR#SYS@imm32@DLL#PR#SUCCESS')
('232140000', 'LoadLibrary#PR#SYS@lpk@DLL#PR#SUCCESS')
('232440000', 'LoadLibrary#PR#SYS@gdi32@DLL#PR#SUCCESS')
('233040000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233060000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233080000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233120000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233130000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233150000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233170000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233180000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233200000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233220000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233230000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233250000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233260000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#SUCCESS')
('233280000', 'LoadLibrary#PR#SYS@kernel32@DLL#PR#S